In [ ]:
from heston_model import HestonModel
from heston_calibration import calibrate
from heston_pricer import Pricer
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
# Heston model params
params = {'kappa': 2.0,'theta': 0.04,'xi': 0.3,'rho': -0.7,'v0': 0.04}
model = HestonModel(params)

# Market params
S0 = 254.43; r = 0.036735; q = 0; T = 1
K = np.linspace(0.5*S0, 2*S0, 20)

# Prices
mc_prices = model.monte_carlo_call(T, S0, r, q, K)
heston_prices =  model.heston_call(T, S0, r, q, K)
fft_prices = model.carr_madan_call(T, S0, r, q, K)
print(f"Max relative error between Carr-Madan and Heston: {np.max(np.abs(fft_prices/heston_prices - 1)):.4e}")

# Plot
plt.plot(K, mc_prices, 'o-', label='Monte Carlo')
plt.plot(K, heston_prices, 's-', label='Heston')
plt.plot(K, fft_prices, '^-', label='Carr-Madan')
plt.xlabel('Strike K')
plt.ylabel('Call Price')
plt.title('Call Prices under Heston Model')
plt.legend()
plt.tight_layout()
plt.grid()
plt.show()

In [ ]:
# Calibration 
file = r"Calibration Template AAPL.xlsx"
surf = pd.read_excel(file)
result = calibrate(model, surf, S0, r, T, q)

In [ ]:
# Vol process
S_t, v_t = model.simulate(S0, T, r, q, npaths = 1000)
plt.plot(np.sqrt(v_t))
plt.tight_layout()
plt.xlabel('Time')
plt.ylabel('Volatility')
plt.title('Simulated Volatility Process')
plt.grid()
plt.show()

In [ ]:
# Pricing 
S0 = 252.14; r = 0.036926; q = 0.004088; T = 1; K = 200; B = 180; n = 1000
pricer = Pricer(model)

# Put Option 
mtm = 7_036.45
put_price = pricer.european(T, S0, r, q, K, type='Put')*n 
print(f"European Put Price: {put_price:.2f}; Relative error: {(put_price/mtm - 1):.3%}")

# Digital Option
mtm = 776.59
digital_price = pricer.digital(T, S0, r, q, K, type='Call')*n
print(f"Digital Call Price: {digital_price:.2f}; Relative error: {(digital_price/mtm - 1):.3%}")

# Barrier Option
mtm = 62_680.25
barrier_price = pricer.barrier(T, S0, r, q, K, B, type='DownAndOut', npaths= 10**5, nsteps= 500, seed = 2025)*n
print(f"Barrier Option Price: {barrier_price:.2f}; Relative error: {(barrier_price/mtm - 1):.3%}")
